In [8]:
import pandas as pd

In [9]:
# List of 12 monthly dataset filenames
dataset_filenames = [
    '201801-PV.CSV',
    '201802-PV.CSV',
    '201803-PV.CSV',
    '201804-PV.CSV',
    '201805-PV.CSV',
    '201806-PV.CSV',
    '201807-PV.CSV',
    '201808-PV.CSV',
    '201809-PV.CSV',
    '201810-PV.CSV',
    '201811-PV.CSV',
    '201812-PV.CSV'
]

In [10]:
# List to store the monthly dataframes
monthly_dataframes = []

In [11]:
# Iterate over each dataset filename
for filename in dataset_filenames:
    # Open the monthly dataset
    dataset = pd.read_csv(filename)
    
    # Drop the original headers in the dataset, use the first row as the new headers
    dataset = dataset.rename(columns=dataset.iloc[0]).drop(dataset.index[0]).reset_index(drop=True)
    
    # Drop the last row ('END OF REPORT') in the dataset
    dataset = dataset.iloc[:-1]
    
    # Drop the 1st-4th, 8th and 10th columns (irrelavant columns:'I','ROOFTOP','ACTUAL','2','QI','LASTCHANGED')
    columns_to_drop = dataset.columns[[0, 1, 2, 3, 7, 9]]
    dataset = dataset.drop(columns_to_drop, axis=1)
    
    # Keep all rows with ‘REGIONID’ as: NSW1, QLD1, SA1, VIC1, TAS1; 
    # Reason: all these Region IDs are the sum of other IDs, such as QLDC, QLDS, QLDN, etc
    regions_to_keep = ['NSW1', 'QLD1', 'VIC1', 'TAS1', 'SA1']
    dataset = dataset[dataset['REGIONID'].isin(regions_to_keep)]
    
    # Keep all records with ‘TYPE’ as: ‘DAILY’; 
    # Reason: ‘DAILY’ is the most accurate results of ‘POWER’, ‘SATELLITE’ is the estimated results of ‘power’)
    dataset = dataset[dataset['TYPE'] == 'DAILY'] 
    
    # Clean the 'REGIONID' column: e.g. 'NSW1' to 'NSW'
    dataset['REGIONID'] = dataset['REGIONID'].replace({'NSW1':'NSW','QLD1':'QLD','VIC1':'VIC','SA1':'SA','TAS1':'TAS'})
    
     # Change POWER to numeric
    dataset['POWER'] = pd.to_numeric(dataset['POWER'], errors='coerce')
    
    # Change INTERVAL_DATETIME to datetime
    dataset['INTERVAL_DATETIME'] = pd.to_datetime(dataset['INTERVAL_DATETIME'])
    
    # Add a new column 'Month'
    dataset['Month'] = int(filename[4:6])  # Extract the month from the filename
    
    # Append the monthly dataset to the list
    monthly_dataframes.append(dataset)

In [12]:
# Concatenate the monthly dataframes vertically to get the yearly electricity generation dataframe
yearly_df = pd.concat(monthly_dataframes, ignore_index=True)
yearly_df

,INTERVAL_DATETIME,REGIONID,POWER,TYPE,Month
0,2018-01-01 00:30:00,NSW,NaN,DAILY,1
1,2018-01-01 00:30:00,QLD,NaN,DAILY,1
2,2018-01-01 00:30:00,SA,NaN,DAILY,1
3,2018-01-01 00:30:00,TAS,NaN,DAILY,1
4,2018-01-01 00:30:00,VIC,NaN,DAILY,1
...,...,...,...,...,...
87595,2019-01-01 00:00:00,NSW,0.0,DAILY,12
87596,2019-01-01 00:00:00,QLD,0.0,DAILY,12
87597,2019-01-01 00:00:00,SA,0.0,DAILY,12
87598,2019-01-01 00:00:00,TAS,0.0,DAILY,12


In [13]:
yearly_df.to_csv('yearly.csv',index=False)

In [14]:
# Delete any rows where the 'INTERVAL_DATETIME' is from 2019
yearly_df = yearly_df[~(yearly_df['INTERVAL_DATETIME'].dt.year == 2019)]
yearly_df

,INTERVAL_DATETIME,REGIONID,POWER,TYPE,Month
0,2018-01-01 00:30:00,NSW,NaN,DAILY,1
1,2018-01-01 00:30:00,QLD,NaN,DAILY,1
2,2018-01-01 00:30:00,SA,NaN,DAILY,1
3,2018-01-01 00:30:00,TAS,NaN,DAILY,1
4,2018-01-01 00:30:00,VIC,NaN,DAILY,1
...,...,...,...,...,...
87590,2018-12-31 23:30:00,NSW,0.0,DAILY,12
87591,2018-12-31 23:30:00,QLD,0.0,DAILY,12
87592,2018-12-31 23:30:00,SA,0.0,DAILY,12
87593,2018-12-31 23:30:00,TAS,0.0,DAILY,12


In [15]:
yearly_df.isnull().sum()

INTERVAL_DATETIME      0
REGIONID               0
POWER                240
TYPE                   0
Month                  0
dtype: int64

In [17]:
# Drop the rows where 'Power' is missing 
# Reason: only the records of 1st Jan 2018 is missing, which is a small amount of data 
yearly_df = yearly_df.dropna(subset=['POWER'])
yearly_df.isnull().sum()

INTERVAL_DATETIME    0
REGIONID             0
POWER                0
TYPE                 0
Month                0
dtype: int64

In [18]:
# 2018 yearly rooftop PV elecrtricity generation aggragation by 'Month' and'REGIONID' 
# Group by 'Month' and 'REGIONID', and sum the 'POWER' - i.e. the power of generator
aggregated_df = yearly_df.groupby(['Month', 'REGIONID'])['POWER'].sum().reset_index()
aggregated_df

,Month,REGIONID,POWER
0,1,NSW,392845.130
1,1,QLD,576868.114
2,1,SA,249090.942
3,1,TAS,34028.350
4,1,VIC,345057.244
5,2,NSW,350448.886
6,2,QLD,437395.013
7,2,SA,216309.739
8,2,TAS,26774.742
9,2,VIC,306602.724


In [20]:
#Add a new column 'Electricity Generation(MWh)'
aggregated_df['Electricity Generation (MWh)'] = round(aggregated_df['POWER'] * (30/60), 2)
aggregated_df

,Month,REGIONID,POWER,Electricity Generation (MWh)
0,1,NSW,392845.130,196422.56
1,1,QLD,576868.114,288434.06
2,1,SA,249090.942,124545.47
3,1,TAS,34028.350,17014.18
4,1,VIC,345057.244,172528.62
5,2,NSW,350448.886,175224.44
6,2,QLD,437395.013,218697.51
7,2,SA,216309.739,108154.87
8,2,TAS,26774.742,13387.37
9,2,VIC,306602.724,153301.36


In [21]:
#Save the aggregated sheet as the yearly aggregated dataset
aggregated_df.to_csv('aggregatedREGIONID_PV_2018.csv', index=False)